In [1]:
%reload_ext autoreload
%autoreload 2

%cd /Users/chompk.visai/Works/cdao/connext/connext-galxe-analytics/

/Users/chompk.visai/Works/cdao/connext/connext-galxe-analytics


In [2]:
import logging

from dotenv import load_dotenv
from omegaconf import OmegaConf

from api.connext import ConnextAPI
from api.constant import Chain
from api.scan import ScanAPI
load_dotenv(".env")

True

In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
cfg = OmegaConf.load("conf/networks.yaml")
api = ConnextAPI(data_dir="data")

In [17]:
txs = api.load_txs()

INFO:root:Loading cache
INFO:root:Cache found, loading cache
INFO:root:Latest block number on ethereum: 16604141
INFO:root:Latest block number on bnb_chain: 25569167
INFO:root:Latest block number on polygon: 39165281
INFO:root:Latest block number on optimism: 73629300
INFO:root:Latest block number on gnosis: 26421076
INFO:root:Latest block number on arbitrum_one: 59924891
INFO:root:Loading transactions from scan API
INFO:root:Number of new transactions on ethereum: 0
INFO:root:Number of new transactions on bnb_chain: 0
INFO:root:Number of new transactions on polygon: 0
INFO:root:Number of new transactions on optimism: 0
INFO:root:Number of new transactions on gnosis: 0
INFO:root:Number of new transactions on arbitrum_one: 0
INFO:root:No new transactions, returning cache


In [29]:
from collections import Counter

{chain: Counter([tx.functionName.split("(")[0] for tx in txs[chain]]).most_common() for chain in api.scan_api.keys()}

{'ethereum': [('xcall', 208),
  ('xcallIntoLocal', 119),
  ('bumpTransfer', 44),
  ('addRouterLiquidityFor', 28),
  ('execute', 23),
  ('approveRouter', 13),
  ('addRelayer', 10),
  ('removeRouterLiquidity', 6),
  ('initializeRouter', 6),
  ('enrollRemoteRouter', 5),
  ('updateLiquidityCap', 5),
  ('removeRouterLiquidityFor', 5),
  ('setupAsset', 4),
  ('assignRoleAdmin', 2),
  ('atInversebrah', 1),
  ('proposeDiamondCut', 1),
  ('addSequencer', 1),
  ('setRelayerFeeVault', 1),
  ('assignRoleWatcher', 1),
  ('proposeNewOwner', 1),
  ('pause', 1),
  ('unpause', 1),
  ('assignRoleRouterAdmin', 1)],
 'bnb_chain': [('xcall', 1053),
  ('addSwapLiquidity', 327),
  ('swap', 250),
  ('removeSwapLiquidity', 50),
  ('bumpTransfer', 48),
  ('xcallIntoLocal', 30),
  ('addRouterLiquidityFor', 29),
  ('execute', 13),
  ('addRelayer', 10),
  ('approveRouter', 10),
  ('initializeRouter', 6),
  ('enrollRemoteRouter', 5),
  ('setupAsset', 4),
  ('removeRouterLiquidity', 4),
  ('initializeSwap', 3),
  ('